<a href="https://colab.research.google.com/github/khivishta/Exercises/blob/main/pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import time
print("Loading files...")
columns = ['TimeStamp', 'Power_kW', 'WindSpeed_mps', 'AmbTemp_DegC', 'BrakeState', 'RotorBrakeAppliedByArls', 'RotorBrakeAppliedByRlcp', 'RotorBrakeAppliedByEStop', 'RlcpHssBrakeApplyPrsnl', 'RlcpRotorLockAppliedByPrsnl',
               'RotorLockFullyEngagedLamp', 'HssBrakeAppliedByRlcpLamp', 'RotorLockPinFullyEngaged1', 'SecsOnline', 'RotorBrakePresHigh2', 'RotorBrakePresHigh1', 'RotorBrakePresLow2', 'RotorBrakePresLow1', 'MeasuredYawError']

start_time = time.time()
df = pd.concat([pd.concat(pd.read_csv(r'/content/drive/MyDrive/LDT-1Hz-Tur-2019-03.csv',usecols=columns,chunksize=1000000)),
                pd.concat(pd.read_csv(r'/content/drive/MyDrive/LDT-1Hz-Tur-2019-04.csv',usecols=columns,chunksize=1000000))])


end_time = (time.time() - start_time)
print("took %s seconds to read the csv files" % end_time)

print("Read all csv files")
print("pickling dataframe...")
start_time = time.time()
df.to_pickle('raw_scada_data.gzip')
end_time = (time.time() - start_time)
print("took %s seconds to pickle the dataframe" % end_time)
print("DONE!")


Loading files...
took 355.1594262123108 seconds to read the csv files
Read all csv files
pickling dataframe...
took 3.7312514781951904 seconds to pickle the dataframe
DONE!


In [ ]:
pd.read_pickle('raw_scada_data.gzip')

,TimeStamp,Power_kW,WindSpeed_mps,BrakeState,AmbTemp_DegC,SecsOnline,MeasuredYawError,RotorBrakeAppliedByArls,RotorBrakeAppliedByRlcp,RotorBrakeAppliedByEStop,RotorLockFullyEngagedLamp,HssBrakeAppliedByRlcpLamp,RlcpHssBrakeApplyPrsnl,RlcpRotorLockAppliedByPrsnl,RotorLockPinFullyEngaged1,RotorBrakePresHigh2,RotorBrakePresHigh1,RotorBrakePresLow2,RotorBrakePresLow1
0,2019-01-01 00:00:00.781,-65.0,12.338920,1.0,12.699990,0.0,-0.029841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,2019-01-01 00:00:01.812,-65.0,12.338920,1.0,12.699990,0.0,-0.029841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2019-01-01 00:00:02.854,-65.0,12.338920,1.0,12.699990,0.0,-0.029841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,2019-01-01 00:00:03.855,-64.0,12.662790,1.0,12.699990,0.0,0.013902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,2019-01-01 00:00:04.866,-64.0,12.662790,1.0,12.699990,0.0,0.013902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370512,2019-02-28 23:59:55.111,-1.0,2.673318,1.0,7.099988,0.0,2.185597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2370513,2019-02-28 23:59:56.132,-1.0,2.706506,1.0,7.099988,0.0,2.195904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2370514,2019-02-28 23:59:57.134,-1.0,2.706506,1.0,7.099988,0.0,2.195904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2370515,2019-02-28 23:59:58.155,-1.0,2.556590,1.0,7.099988,0.0,2.200817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [9]:
import pandas as pd
import os
import glob
import numpy as np
import sklearn
from numpy.random import seed
import time

print("Reading the pickle file...")

start_time = time.time()
df = pd.read_pickle('/content/drive/MyDrive/raw_scada_data.gzip')
end_time = (time.time() - start_time)
print("took %s seconds to read the pickle file" % end_time)

print("pickle file read successfully")

start_time = time.time()
df = df[((df['SecsOnline'] == 1))]
df = df[((df['RotorBrakeAppliedByArls'] == 0))]
df = df[((df['RotorBrakeAppliedByEStop'] == 0))]
df = df[((df['RotorBrakeAppliedByRlcp'] == 0))]
df = df[((df['RlcpHssBrakeApplyPrsnl'] == 0))]
df = df[((df['RlcpRotorLockAppliedByPrsnl'] == 0))]
df = df[((df['HssBrakeAppliedByRlcpLamp'] == 0))]
df = df[((df['BrakeState'] == 1))]
df = df[((df['RotorBrakePresHigh2'] == 0))]
df = df[((df['RotorBrakePresHigh1'] == 0))]
df = df[((df['RotorBrakePresLow2'] == 1))]
df = df[((df['RotorBrakePresLow1'] == 1))]
df = df[((df['RotorLockPinFullyEngaged1'] == 0))]
df = df[~((df['WindSpeed_mps'] > 20) & (df['Power_kW'] < 6000))]
df = df[~(df['WindSpeed_mps'] > 16.5)]

q75, q25 = np.percentile(df.loc[:, 'WindSpeed_mps'], [75, 25])
IQR = q75-q25
upper_bound = q75+(1.5*IQR)
lower_bound = q25-(1.5*IQR)

df = df[~((df['WindSpeed_mps'] <= lower_bound) |
            (df['WindSpeed_mps'] >= upper_bound))]
df = df[~(df['Power_kW'] < 0)]

df = df[['TimeStamp',	'Power_kW',	'WindSpeed_mps','AmbTemp_DegC']]

end_time = (time.time() - start_time)
print("took %s seconds to do all the cleaning steps" % end_time)

print("pickling dataframe...")
start_time = time.time()
df.to_pickle('/content/drive/MyDrive/cleaned_scada_data.gzip')
end_time = (time.time() - start_time)
print("took %s seconds to pickle the dataframe" % end_time)
print("DONE!")

print('all done')

Reading the pickle file...
took 2.229395866394043 seconds to read the pickle file
pickle file read successfully
took 5.9744956493377686 seconds to do all the cleaning steps
pickling dataframe...
took 2.048758029937744 seconds to pickle the dataframe
DONE!
all done
